# iris数据集内容

In [1]:
from sklearn.datasets import load_iris

iris = load_iris()
print(iris.keys())

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])


In [7]:
print(iris['feature_names'])
print(iris['data'][:5])

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]


In [11]:
print(iris['target_names'])
print(iris['target'][40:60])

['setosa' 'versicolor' 'virginica']
[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1]


# 为卫星数据集训练并微调一个决策树

In [27]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.tree import DecisionTreeClassifier

#获取数据集
x,y = make_moons(n_samples=10000,noise=0.4)

#拆分数据集
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state = 42)
print(len(x_train),len(x_test))

8000 2000


In [28]:
#使用交叉验证的网格搜索获取最佳模型参数
param_grid = {'max_leaf_nodes':list(range(2,100)),'min_samples_split':[2,3,4]}
dts_cfg = DecisionTreeClassifier(random_state=42)
gridsearch = GridSearchCV(dts_cfg,param_grid,n_jobs=-1,verbose=1,cv =3)
gridsearch.fit(x_train,y_train)

Fitting 3 folds for each of 294 candidates, totalling 882 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done 882 out of 882 | elapsed:    1.6s finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], 'min_samples_split': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       s

In [29]:
gridsearch.best_estimator_

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=4, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=42, splitter='best')

In [30]:
from sklearn.metrics import accuracy_score
y_test_pred = gridsearch.predict(x_test)
accuracy_score(y_test,y_test_pred)

0.8595

In [38]:
from sklearn.model_selection import ShuffleSplit
n_trees = 1000
n_instances = 100
mini_sets = []

ss = ShuffleSplit(n_splits=n_trees,test_size=len(x_train)- n_instances,random_state = 42)
for mini_train_index,mini_test_index in ss.split(x_train):
    x_mini_train = x_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((x_mini_train,y_mini_train))

In [40]:
from sklearn.base import clone
import numpy as np

forest = [clone(gridsearch.best_estimator_) for _ in range(n_trees)]
accuracy_scores = []

for tree,(x_mini_train,y_mini_train) in zip(forest,mini_sets):
        tree.fit(x_mini_train,y_mini_train)
        y_pred = tree.predict(x_test)
        accuracy_scores.append(accuracy_score(y_test,y_pred))
        
np.mean(accuracy_scores)

0.8268355000000001

对于每个测试集实例，生成1000个决策树的预测，并只保留最频繁的预测(您可以为此使用SciPy的mode()函数)。这给出了测试集的多数投票预测

In [46]:
y_pred = np.empty([n_trees, len(x_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    y_pred[tree_index] = tree.predict(x_test)

In [47]:
from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(y_pred, axis=0)

In [48]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.8575